# Convolutional Neural Network in Keras

Bulding a Convolutional Neural Network to classify MNIST digits.

#### Set seed for reproducibility

In [ ]:
import numpy as np
np.random.seed(42)

#### Load dependencies

In [ ]:
import os
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, LeakyReLU, BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from keras_contrib.layers.advanced_activations import SineReLU

#### Load data

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

#### Preprocess data
Flatten and normalise input data.

In [ ]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

X_train = X_train.astype("float32")/255.
X_test = X_test.astype("float32")/255.

In [ ]:
# One-hot encoded categories
n_classes = 10
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

#### Design Neural Network architecture

In [ ]:
epsilon_cnn = 0.0025
epsilon_dense = epsilon_cnn

model = Sequential()

model.add(Conv2D(32, 7, padding = 'same', input_shape = (28, 28, 1)))
model.add(SineReLU(epsilon_cnn))

model.add(Conv2D(32, 7, padding = 'same'))
model.add(SineReLU(epsilon_cnn))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.20))

model.add(Conv2D(64, 3, padding = 'same'))
model.add(SineReLU(epsilon_cnn))

model.add(Conv2D(64, 3, padding = 'same'))
model.add(SineReLU(epsilon_cnn))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.30))

model.add(Conv2D(128, 2, padding = 'same'))
model.add(SineReLU(epsilon_cnn))

model.add(Conv2D(128, 2, padding = 'same'))
model.add(SineReLU(epsilon_cnn))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(512))
model.add(SineReLU(epsilon_dense))
model.add(Dropout(0.50))

model.add(Dense(10, activation = "softmax"))

model.summary()

#### Callbacks

In [ ]:
modelCheckpoint = ModelCheckpoint(monitor='val_acc', filepath='model_output/weights-cnn-mnist.hdf5',
                                               save_best_only=True, mode='max')
earlyStopping = EarlyStopping(monitor='val_acc', mode='max', patience=20)


if not os.path.exists('model_output'):
    os.makedirs('model_output')

tensorboard = TensorBoard("../logs/convnet-mnist-native-relus-III")

#### Configure model

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#### Train!

In [ ]:
history = model.fit(X_train, y_train, batch_size = 128, epochs = 50, verbose = 1,
          validation_split = 0.1, callbacks=[modelCheckpoint, earlyStopping, tensorboard])

#### Test Predictions

In [ ]:
saved_model = keras.models.load_model('model_output/weights-cnn-mnist.hdf5')
predictions = saved_model.predict_classes(X_test, verbose = 2)
np.std(history.history['loss'])

#### Test Final Accuracy

In [ ]:
final_loss, final_acc = saved_model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))
final_loss, final_acc = model.evaluate(X_test, y_test, verbose = 1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
#Final loss: 0.0197, final accuracy: 0.9950 -> SineReLU (0.0045, 0.0083); STD loss => 0.03690133793565328
#Final loss: 0.0159, final accuracy: 0.9944 -> SineReLU (0.0025); STD loss => 0.04272104275245348
#Final loss: 0.0192, final accuracy: 0.9945 -> SineReLU (0.0050, 0.0083); STD loss => 0.04411841413608315
#Final loss: 0.0212, final accuracy: 0.9942 -> SineReLU (0.0030, 0.0083); STD loss => 0.037860394162356176

#Final loss: 0.0203, final accuracy: 0.9939 -> ReLU; STD loss => 0.04592196838390996